In [ ]:
import cv2  # NOQA (Must import before importing caffe2 due to bug in cv2)
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import numpy as np

from caffe2.python import workspace

%matplotlib inline

import detectron.datasets.dummy_datasets as dummy_datasets
import detectron.utils.c2 as c2_utils
import detectron.utils.vis as vis_utils


c2_utils.import_detectron_ops()
c2_utils.import_custom_ops()

# OpenCL may be enabled by default in OpenCV3; disable it because it's not
# thread safe and causes unwanted GPU memory allocations.
cv2.ocl.setUseOpenCL(False)

workspace.GlobalInit(['caffe2', '--caffe2_log_level=0'])

dummy_mot_dataset = dummy_datasets.get_mot_dataset()

from detectron.datasets.json_dataset import JsonDataset
def get_roidb(dataset_name):
    ds = JsonDataset(dataset_name)
    roidb = ds.get_roidb(gt=True)
    return roidb
roidb = get_roidb('mot_2017_overtrain')

im_list = [cv2.imread(im_data['image']) for im_data in roidb]

show_class = False
show_track = True

track_gt = np.array([float(id_one == id_two) for id_one in roidb[0]['track_ids'] for id_two in roidb[1]['track_ids']])
im_one_gt, im_two_gt = vis_utils.vis_image_pair_opencv_gt(
    [im.copy() for im in im_list],
    [im_data['gt_classes'] for im_data in roidb],
    [im_data['boxes'] for im_data in roidb],
    [None] * len(roidb),
    [None] * len(roidb),
    track_gt,
    dataset=dummy_mot_dataset,
    show_class=show_class,
    show_track=show_track,
    show_box=True,
)

ax = vis_utils.get_ax(2, shape=im.shape, title="Ground truth")
ax[0].imshow(cv2.cvtColor(im_one_gt, cv2.COLOR_BGR2RGB))
ax[1].imshow(cv2.cvtColor(im_two_gt, cv2.COLOR_BGR2RGB))
plt.show()